In [1]:
import tensorflow as tf
import pprint
import pycurl
from io import BytesIO
import json
import time

/home/hudz0r/miniconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Load model from file

In [96]:
sess = tf.Session()
# we want the variable values from from my_tf_model (not working right now)
saver = tf.train.import_meta_graph('models/my_tf_model.meta')
saver.restore(sess, tf.train.latest_checkpoint('models'))

graph = tf.get_default_graph()

def conv_net_old(x,reuse=False, dropout=0.1610, kernel_size=3, filters=32, hiddenunits=500):
    # Define a scope for reusing the variables
    # with tf.variable_scope('model', reuse=reuse): is this needed?
    conv1    = tf.layers.conv2d(inputs=x,
                                filters=filters,
                                kernel_size=kernel_size,
                                activation=tf.nn.relu)
    
    maxpool1 = tf.layers.max_pooling2d(inputs=conv1,
                                       pool_size=2,
                                       strides=2)
    
    conv2    = tf.layers.conv2d(inputs=maxpool1,
                                filters=filters,
                                kernel_size=kernel_size,
                                activation=tf.nn.relu)
    maxpool2 = tf.layers.max_pooling2d(inputs=conv2,
                                       pool_size=2,
                                       strides=2)
    conv3    = tf.layers.conv2d(inputs=maxpool2,
                                filters=filters,
                                kernel_size=kernel_size,
                                activation=tf.nn.relu)

    maxpool3 = tf.layers.max_pooling2d(inputs=conv3,
                                       pool_size=2,
                                       strides=2)

    flat     = tf.contrib.layers.flatten(inputs=maxpool3)
        
    dropout1 = tf.layers.dropout(inputs=flat,
                                 rate=dropout,
                                 training=True)
    
    dense1   = tf.layers.dense(inputs=dropout1,
                               units=hiddenunits,
                               activation=tf.nn.relu)

    print(dropout1)
    
    dropout2 = tf.layers.dropout(inputs=dense1,
                                 rate=dropout,
                                 training=True)
    out = tf.layers.dense(dropout2,
                          units=1)

    return out

# init variables from trained model

W_conv1 = graph.get_collection('trainable_variables')[0]
b_conv1 = graph.get_collection('trainable_variables')[1]
W_conv2 = graph.get_collection('trainable_variables')[2]
b_conv2 = graph.get_collection('trainable_variables')[3]
W_conv3 = graph.get_collection('trainable_variables')[4]
b_conv3 = graph.get_collection('trainable_variables')[5]
W_dense1 = graph.get_collection('trainable_variables')[6]
b_dense1 = graph.get_collection('trainable_variables')[7]
W_out = graph.get_collection('trainable_variables')[8]
b_out = graph.get_collection('trainable_variables')[9]

# build graph

def build_conv_net(dropout=0.1610, kernel_size=3, filters=32, hiddenunits=500):
    
    pkeep = tf.placeholder(tf.float32)
    
    X = tf.placeholder(tf.float32, [1,112, 200, 3])
    
    conv1 = tf.nn.relu(tf.nn.conv2d(X, W_conv1, strides=[1,1,1,1], padding="VALID") + b_conv1)
 
    maxpool1 = tf.layers.max_pooling2d(inputs=conv1,
                                       pool_size=2,
                                       strides=2)
        
    conv2 = tf.nn.relu(tf.nn.conv2d(maxpool1, W_conv2, strides=[1,1,1,1], padding="VALID") + b_conv2)
    
    maxpool2 = tf.layers.max_pooling2d(inputs=conv2,
                                       pool_size=2,
                                       strides=2)
    
    conv3 = tf.nn.relu(tf.nn.conv2d(maxpool2, W_conv3, strides=[1,1,1,1], padding="VALID") + b_conv3)
    
    
    maxpool3 = tf.layers.max_pooling2d(inputs=conv3,
                                       pool_size=2,
                                       strides=2)
            
    flat = tf.contrib.layers.flatten(inputs=maxpool3)
            
    dropout1 = tf.nn.dropout(flat, pkeep)
    
    print(dropout1)
    
    dense1 = tf.nn.relu(tf.matmul(dropout1, W_dense1) + b_dense1)
    
    dropout2 = tf.nn.dropout(dense1, pkeep)
    
    out = tf.nn.relu(tf.matmul(dropout2, W_out) + b_out)

    return out


INFO:tensorflow:Restoring parameters from models/my_tf_model


### Graph info

In [97]:
print(graph.get_all_collection_keys())
print(graph.get_collection('trainable_variables')[8])

['queue_runners', 'summaries', 'trainable_variables', 'train_op', 'variables', 'update_ops']
<tf.Variable 'dense_1/kernel:0' shape=(500, 1) dtype=float32_ref>


### Functions

In [98]:
# returns prediction of number of cars from img of dimensions 200x112x3
def count_from_image(img):
    print("build graph")
    c = conv_net(img, reuse=True)
    print("done")
    return c

# returns image from webcam with id
def get_img(webcam_id):
    image = tf.read_file("data/test_image.jpg")
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [112, 200])
    return tf.expand_dims(image,0)

    

### Main program

In [99]:
webcam_id_list = [33] #dummy values
sleep_time = 60 #check every minute
while(1):
    for w in webcam_id_list:
        i = get_img(w)
        
        conv_net_old(i)
        build_conv_net()
        
        
        g = count_from_image(i)
        sess.run(tf.global_variables_initializer())
        # forward prop the img and recieve prediction
        count = sess.run(g)
        print("Webcam " + str(w) + " has " + str(count[[0]]) + " number of cars")
    time.sleep(sleep_time)

Tensor("dropout_68/dropout/mul:0", shape=(1, 8832), dtype=float32)
Tensor("dropout_70/mul:0", shape=(1, 8832), dtype=float32)


ValueError: Dimensions must be equal, but are 8832 and 39936 for 'MatMul_23' (op: 'MatMul') with input shapes: [1,8832], [39936,500].